# Ramia_Assignment4

In this project I will use all explanatory variables (with the exception of neighborhood) and all 506 census tract observations from the Boston Housing Study to advise a real estate brokerage firm in its attempt to employ machine learning methods. With the median value of homes in thousands of 1970 dollars as my target variable, I will attempt to assess the market value of residential real estate. I will do so by employing three modeling methods: linear regression, ridge regression, and random forests. Evaluating these methods within a cross-validation design using root mean-squared error (RMSE) as an index of prediction error, I discovered that random forests outperformed both regression models.

In an attempt to improve on the random forest model, I employed the grid search technique for optimizing hyperparameters. I also explored gradient boosting as another potential improvement on the random forest model. In the end, the gradient boosting model performed the best, being able to predict median value of homes with an accuracy of ~0.98 for the training set and ~0.93 for the test set.

**Part 1: Data Prep and Exploration**

Seed value for random number generators to obtain reproducible results:

In [1]:
RANDOM_SEED = 1

Although I standardize X and y variables on input, I will fit the intercept term in the models. Expect fitted values to be close to zero.

In [2]:
SET_FIT_INTERCEPT = True

Import base packages into the namespace for this program:

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

Import modeling routines from Scikit Learn packages:

In [4]:
import sklearn.linear_model 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from math import sqrt  # for root mean-squared error calculation
from sklearn.ensemble import RandomForestRegressor

Read data for the Boston Housing Study, creating data frame boston_input:

In [5]:
boston_input = pd.read_csv('boston.csv')

Drop neighborhood from the data being considered:

In [6]:
boston = boston_input.drop('neighborhood', 1)

Check the data:

In [7]:
print('\nFirst 5 Rows of Data:\n')
print(boston.head())

print('\nGeneral Description:\n')
print(boston.info())

print('\nDescriptive Statistics:\n')
print(boston.describe())


First 5 Rows of Data:

      crim    zn  indus  chas    nox  rooms   age     dis  rad  tax  ptratio  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242     17.8   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242     17.8   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222     18.7   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222     18.7   

   lstat    mv  
0   4.98  24.0  
1   9.14  21.6  
2   4.03  34.7  
3   2.94  33.4  
4   5.33  36.2  

General Description:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
crim       506 non-null float64
zn         506 non-null float64
indus      506 non-null float64
chas       506 non-null int64
nox        506 non-null float64
rooms      506 non-null float64
age        506 non-null float64
dis        506 non-null float64
rad        

Set up preliminary data for fitting the models. The last column is the median housing value response. The preceding columns are the explanatory variables.

In [8]:
model_data = boston.values

The model data will be a standardized form of the preliminary model data. Standard scalers subtract the mean value (so standardized values always have a zero mean), and then it divides by the standard deviation so that the resulting distribution has unit variance.

It is important to transform the data so that all attributes are on a common scale (e.g. using a standard scaler) before performing Ridge Regression, as it is sensitive to the scale of the input features. This is true of most regularized models.

As with all transformations, it is important to fit the scalers to the training data only, not to the full dataset (including the test set). Only then can you use them to transform the training set and the test set (and new data). I use pipeline to ensure that this is done correctly below.

In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

**Part 2: Model Evaluation**

Specify the set of models being evaluated. Here I will explore three models: linear regression, ridge regression, and random forests.

I chose ridge regression over similar regularized regression methods such as lasso and ElasticNet. In practice, ridge regression is usually the first choice between these three models. Lasso and ElasticNet have their benefits but ridge regression is usually the widely regarded default.

Further, the ridge model makes a trade-off between the simplicity of the model and its performance on the training set. How much importance the model places on simplicity versus training set performance can be specified by the user, using the alpha parameter. The default parameter alpha=1, however, the optimum setting of alpha depends on the particular dataset being used. Increasing alpha forces coefficients to move more toward zero, which decreases training set performance but might help generalization. Decreasing alpha allows the coefficients to be less restricted, and we end up with a model that resembles linear regression. However, this is at the cost of generalizability. Here I will use an alpha of 10, since that performed best in my last study.

Finally, I set normalize=False because I standardize the model input data outside of the modeling method calls.

In [10]:
names = ['Linear_Regression', 'Ridge_Regression', 'Random_Forest']

regressors = [LinearRegression(fit_intercept = SET_FIT_INTERCEPT,
                               normalize = False), 
              Ridge(alpha = 10, solver = 'cholesky',
                    fit_intercept = SET_FIT_INTERCEPT,
                    normalize = False,
                    random_state = RANDOM_SEED),
              RandomForestRegressor(n_estimators = 100,
                                    random_state = RANDOM_SEED)
             ]

Specify the k-fold cross-validation design:

In [11]:
from sklearn.model_selection import KFold

Ten-fold cross-validation is employed here.

In [12]:
N_FOLDS = 10

Set up numpy array for storing results.

In [13]:
cv_results = np.zeros((N_FOLDS, len(names)))

kf = KFold(n_splits = N_FOLDS, shuffle=False, random_state = RANDOM_SEED)

Check the splitting process by looking at fold observation counts:

In [14]:
index_for_fold = 0  # fold count initialized 
for train_index, test_index in kf.split(model_data):
    X_train = model_data[train_index, 0:-1]
    X_test = model_data[test_index, 0:-1]
    y_train = model_data[train_index, -1]
    y_test = model_data[test_index, -1]   
    
    index_for_method = 0  # initialize
    for name, reg_model in zip(names, regressors):
        pipeline = make_pipeline(StandardScaler(),
                                 reg_model)
        pipeline.fit(X_train, y_train)  # fit on the train set for this fold
        y_test_predict = pipeline.predict(X_test) # evaluate on the test set for this fold
        fold_method_result = sqrt(mean_squared_error(y_test, y_test_predict))
        cv_results[index_for_fold, index_for_method] = fold_method_result
        index_for_method += 1
  
    index_for_fold += 1

cv_results_df = pd.DataFrame(cv_results)
cv_results_df.columns = names

In [15]:
print('\n----------------------------------------------')
print('Average results from ', N_FOLDS, '-fold cross-validation\n',
      'in standardized units (mean 0, standard deviation 1)\n',
      '\nMethod               Root mean-squared error', sep = '')     
print(cv_results_df.mean())


----------------------------------------------
Average results from 10-fold cross-validation
in standardized units (mean 0, standard deviation 1)

Method               Root mean-squared error
Linear_Regression    5.154728
Ridge_Regression     5.060819
Random_Forest        4.179909
dtype: float64


Based on the results of the 10-fold cross-validation, it appears as though the random forest model provides the lowest root mean squared error (RMSE) among all of the models tested. This model is able to predict the target variable (mv) with a 4.18 RMSE, on average.

Although the random forest was superior, I believe there is potential to further improve it's accuracy by optimizing hyperparameters. I will explore this below using random grid search.

In [16]:
from sklearn.model_selection import RandomizedSearchCV

Create a random grid by defining the options for:
    1) Number of trees in random forest
    2) Number of features to consider at every split
    3) Maximum number of levels in tree
    4) Minimum number of samples required to split a node
    5) Minimum number of samples required at each leaf node
    6) Method of selecting samples for training each tree (This will always be True)

In [17]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 9)]
max_features = [int(x) for x in np.linspace(start = 1, stop = 12, num = 12)]
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450, 500]}


Use the random grid to search for best hyperparameters using 10 fold cross validation:

In [18]:
from sklearn.model_selection import train_test_split

X = model_data[:,0:-1]
y = model_data[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_SEED)

# First create the base model to tune
rf = RandomForestRegressor()

# Next search across 100 different combinations and use all available cores
rf_random = RandomizedSearchCV(estimator = rf,
                               param_distributions = random_grid,
                               n_iter = 100,
                               cv = 10,
                               verbose = 2,
                               random_state = RANDOM_SEED,
                               n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   49.4s finished
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450, 500], 'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True]},
          pre_dispatch='2*n_jobs', random_state=1, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

View the best parameters from fitting the random search:

In [19]:
rf_random.best_params_

{'n_estimators': 500,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 5,
 'max_depth': 60,
 'bootstrap': True}

To determine if random search yielded a better model, I compare the base model with the best random search model:

In [20]:
base_model = RandomForestRegressor(n_estimators = 100, random_state = RANDOM_SEED)
base_model.fit(X_train, y_train)
print("Base Model Accuracy")
print("Training: {:.3f}".format(base_model.score(X_train, y_train)))
print("Test: {:.3f}".format(base_model.score(X_test, y_test)),"\n")

best_random = rf_random.best_estimator_
print("Best Random Accuracy")
print("Training: {:.3f}".format(best_random.score(X_train, y_train)))
print("Test: {:.3f}".format(best_random.score(X_test, y_test)))

Base Model Accuracy
Training: 0.980
Test: 0.904 

Best Random Accuracy
Training: 0.984
Test: 0.903


I achieved an unspectacular improvement in accuracy on the training data of 0.004 and actually decreased in accuracy on the test data! Further, it appears as though both of these models are overfitting the training data. Hopefully gradient boosting can improve on these results.

In [21]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(random_state=RANDOM_SEED)
gbrt.fit(X_train, y_train)
print("Gradient Boosting Accuracy")
print("Training: {:.3f}".format(gbrt.score(X_train, y_train)))
print("Test: {:.3f}".format(gbrt.score(X_test, y_test)))

Gradient Boosting Accuracy
Training: 0.980
Test: 0.926


While it appears as though overfitting may still be a slight issue, I think it's safe to conclude that gradient boosting is the model that should be recommended to the real estate agency.

Let's take a look at the variables that the gradient boosting model determined to be the most important in predicting median value of homes.

In [22]:
feature_importances = pd.DataFrame(gbrt.feature_importances_,
                                   index = boston.columns[0:-1],
                                   columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

         importance
lstat      0.480762
rooms      0.286707
dis        0.097327
ptratio    0.045325
nox        0.041528
crim       0.018002
tax        0.012792
age        0.012352
indus      0.003599
rad        0.000981
zn         0.000425
chas       0.000199


Using these variables and the models demonstrated here, the real estate agency will be able to make smarter investments in the Boston housing market.